In [1]:
import pandas as pd
import torch

diabetes_df = pd.read_csv("diabetes.csv")
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = diabetes_df.drop('Outcome', axis=1).values
y = diabetes_df['Outcome'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42, stratify=y)

# #Standardize
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)
# print(X_train)
# print(y_train)

In [3]:
import torch.nn as nn
import torch.nn.functional as F #where the activation functions are

#create tensors = matrices 
X_train = torch.FloatTensor(X_train) 
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

tensor([[ 0.9314,  2.0179,  0.7807,  ...,  0.4315, -0.3748,  0.6321],
        [ 0.6326, -1.1486,  0.4654,  ..., -0.1198, -0.2942,  0.7170],
        [-0.5625, -0.4769, -0.2703,  ..., -0.2096,  2.7452,  0.0381],
        ...,
        [-0.8613, -0.7648,  0.0450,  ...,  0.7648, -0.7838, -0.3014],
        [ 0.6326,  2.2099,  1.2010,  ...,  0.4315, -0.6047,  2.7537],
        [ 0.0351,  0.7385, -0.5856,  ..., -0.3378, -0.5778,  0.2927]])


In [4]:
#artificial neural network
class ANN_Model(nn.Module):
    def __init__(self, input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__() #super is a computed indirect reference. So, it isolates changes
        # and makes sure that children in the layers of multiple inheritence are calling
        #the right parents
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
        
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [5]:
torch.manual_seed(42)

#create instance of model
ann = ANN_Model()

In [18]:
#loss function
loss_function = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.AdamX(ann.parameters(),lr=0.01)

In [19]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = ann.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss}')
        
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() 
    optimizer.step() #perform one optimization step each epoch

Epoch number: 1 with loss: 0.009586000815033913
Epoch number: 11 with loss: 0.009570097550749779
Epoch number: 21 with loss: 0.009559844620525837
Epoch number: 31 with loss: 0.009551357477903366
Epoch number: 41 with loss: 0.009544191882014275
Epoch number: 51 with loss: 0.009537060745060444
Epoch number: 61 with loss: 0.009530364535748959
Epoch number: 71 with loss: 0.009523759596049786
Epoch number: 81 with loss: 0.009517444297671318
Epoch number: 91 with loss: 0.009511386044323444
Epoch number: 101 with loss: 0.009505296126008034
Epoch number: 111 with loss: 0.009499643929302692
Epoch number: 121 with loss: 0.009493684396147728
Epoch number: 131 with loss: 0.009488378651440144
Epoch number: 141 with loss: 0.009483672678470612
Epoch number: 151 with loss: 0.009477845393121243
Epoch number: 161 with loss: 0.009472950361669064
Epoch number: 171 with loss: 0.009467746131122112
Epoch number: 181 with loss: 0.009463030844926834
Epoch number: 191 with loss: 0.009458061307668686
Epoch numbe

In [20]:
#predictions
y_pred = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        prediction = ann(data)
        y_pred.append(prediction.argmax()) 

In [21]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.79      0.77       150
           1       0.56      0.51      0.53        81

    accuracy                           0.69       231
   macro avg       0.65      0.65      0.65       231
weighted avg       0.68      0.69      0.68       231



AdamX performs the similary like Adam since  Adamax algorithmis a variant of Adam based on infinity norm.

In [42]:
def dv_count(num):
    dv = []
    
    for i in range(1,num+1):
        if num%i==0:
            dv.append(i)
            
    if len(dv)==1:
        return "there" + " is " + str(len(dv)) + ": " +" ".join(str(e) for e in dv)
    else:
        return "there" + " are " + str(len(dv)) + ": " + " ".join(str(dv[i]) for i in range(len(dv)) if i != len(dv)- 1 else str("and" + str(dv[i]))
    
dv_count(5)

SyntaxError: invalid syntax (<ipython-input-42-f70a38606484>, line 11)